In [ ]:
import geopandas
import pandas as pd
import re
import urllib

from library import DataLibrary

data_directory = '../data'
data_library_filename = 'library.csv'

library = DataLibrary( data_directory, data_library_filename )

In [ ]:
build_cache = False

def append_query_to_url ( url, offset = 0 ):
    return f'{url}?where=1%3D1&timeRelation=esriTimeRelationOverlaps&geometryType=esriGeometryEnvelope&spatialRel=esriSpatialRelIntersects&units=esriSRUnit_Foot&returnGeometry=true&returnTrueCurves=false&returnIdsOnly=false&returnCountOnly=false&returnZ=false&returnM=false&returnDistinctValues=false&returnExtentOnly=false&sqlFormat=none&featureEncoding=esriDefault&f=geojson&resultOffset={offset}'

def fetch_geojson ( url, append_query = True ):
    offset = 0
    geo_df = geopandas.GeoDataFrame()
    while True:
        if append_query:
            offset_url = append_query_to_url( url, offset = offset )

        response = urllib.request.urlopen( offset_url, {} )
        json = response.read()
        request_df = geopandas.read_file( json )
        if not append_query or len( request_df ) == 0:
            return geo_df

        geo_df = pd.concat(
            [
                geo_df,
                request_df
            ],
            axis = 0,
            ignore_index = True
        ).pipe( geopandas.GeoDataFrame )

        offset += len( request_df )

if build_cache:
    library.clear_library()

    # road data (index 0)
    url_road_data = 'https://gisweb.ci.roswell.ga.us/arcgis/rest/services/LGIM_ReferenceData/RoadCenterline/MapServer/0/query'
    # water
    url_water_data = 'https://gisweb.ci.roswell.ga.us/arcgis/rest/services/LGIM_ReferenceData/Waterbody/MapServer/0/query'
    # streams
    url_streams_data = 'https://gisweb.ci.roswell.ga.us/arcgis/rest/services/LGIM_ReferenceData/Streams_110603_/MapServer/0/query'

    urls = {
        'road': url_road_data,
        'water': url_water_data,
        'streams': url_streams_data
    }

    for label, url in urls.items():
        geojson = fetch_geojson( url )

        if len( geojson ) > 0:
            library.save_data( geojson, label = label )

        else:
            print( f'> No data returned for {label}' )

    # build the park layers
    park_data_directory_url = 'https://gisweb.ci.roswell.ga.us/arcgis/rest/services/LGIM_Parks/Park_Facilities/MapServer'
    response = urllib.request.urlopen( park_data_directory_url )
    park_data_directory_html = response.read()
    park_data_layers = re.findall( r'\/(\d+)">([^<]+)<\/a', park_data_directory_html.decode() )

    for ix, label in park_data_layers:
        park_layer_url = f'https://gisweb.ci.roswell.ga.us/arcgis/rest/services/LGIM_Parks/Park_Facilities/MapServer/{ix}/query'
        geojson = fetch_geojson( park_layer_url )

        if len( geojson ) > 0:
            library.save_data( geojson, label = 'park', layer = label, ix = ix )

        else:
            print( f'> No data returned for {label}' )
